# tweet grab

In [1]:
import os
import tweepy
import logging
import pprint
import json
import time
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

print('imported modules successfully.')

imported modules successfully.


In [2]:
try:
    import boto3
    import botocore.session
    from botocore.exceptions import ClientError
    from boto3.dynamodb.conditions import Key, Attr
    print("imported boto3.")

except Exception as e:
    print(e)

imported boto3.


In [3]:
! pip3 install ../main/dist/twitter-0.1.0-py3-none-any.whl --force-reinstall

Processing /home/steven/Documents/twitter.ai/main/dist/twitter-0.1.0-py3-none-any.whl
  Attempting uninstall: twitter
    Found existing installation: twitter 0.1.0
    Uninstalling twitter-0.1.0:
      Successfully uninstalled twitter-0.1.0


In [4]:
# import twitter

### set api creds

In [19]:
base = os.getcwd()
data_dir = os.path.join(base, 'data')
log_dir = os.path.join(base, 'logs')
pp = pprint.PrettyPrinter(indent = 1)

os.chdir(base)

with open('./../creds.json', 'r') as f:
    tweepy_creds = json.load(f)
f.close()
    
auth = tweepy.OAuthHandler(tweepy_creds['twitter-api-key'], tweepy_creds['twitter-secret-key'])
auth.set_access_token(tweepy_creds['twitter-access-token'], tweepy_creds['twitter-secret-access'])

api = tweepy.API(auth)
logging.debug('authorized API w/ client id and secret.')


In [20]:
try:
  client = boto3.client('dynamodb',
    endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]),
    aws_access_key_id=tweepy_creds["aws-access-key"],
    aws_secret_access_key=tweepy_creds["aws-secret-access-key"],
    region_name='us-east-2')

  dynamodb = boto3.resource('dynamodb', \
              endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]), \
              region_name='us-east-2')

  session = botocore.session.get_session()
  dynamodb_session = session.create_client('dynamodb', \
                      region_name='us-east-2',
                      endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]),
                      aws_access_key_id=tweepy_creds["aws-access-key"],
                      aws_secret_access_key=tweepy_creds["aws-secret-access-key"]) # low-level client

  tweets = dynamodb.Table("tweets")

  print("created dynamo client.")

except Exception as e:
  print(e)

created dynamo client.


### get followers

In [21]:
def get_followers(user):
    following = []
    friends_response = api.get_followers(screen_name = user, count = 100, cursor = -1)
    following.extend([t.screen_name for t in friends_response[0]])
    next_cursor = friends_response[1][1]
    print("got most recent 100 following")

    while next_cursor != 0:
        try:
            friends_response = api.get_followers(screen_name = user, count = 100, cursor = next_cursor)
            following.extend([t.screen_name for t in friends_response[0]])
            next_cursor = friends_response[1][1]
            print("got next 100 following")
        
        except:
            time.sleep(60*15)

    return(following)

In [22]:
try:
    my_following = get_followers("xanax_princess_")

except Exception as e:
    print("Exception: {}".format(e))

got most recent 100 following
got next 100 following


In [23]:
print(len(my_following))

198


In [24]:
# test = api.user_timeline(screen_name = my_following[0], count = 25)

In [25]:
# m = len(test)
# test[m-1]._json["id"]

In [26]:
def batch_put_response(response, table):
    with table.batch_writer(overwrite_by_pkeys=["user", "created_at"]) as batch:
        for r in response:
            try:
                batch.put_item(Item = {
                    "user": r.user.screen_name,
                    "created_at": r.created_at.strftime("%Y-%m-%dT%H:%M:%SZ"),
                    "tweet": r.text,
                    "id": r.id,
                    "retweets": r.retweet_count,
                    "favorites": r.favorite_count,
                    "inserted_at": dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
                })
        
            except ClientError as err:
                print("Couldn't load data into table {}. Here's why: {}: {}".format(table.name, err.response['Error']['Code'], err.response['Error']['Message']))
                raise

### get tweets per user

In [27]:
def get_tweets(user, table):
    print("scraping tweets for user: {}".format(user))
    timeline_response = api.user_timeline(screen_name = user, count = 25)
    n = len(timeline_response)
    last_id = timeline_response[n-1]._json["id"]
    i = 0

    print("putting first batch")
    batch_put_response(timeline_response, table)
    print("inserted first batch to dynamodb successfully.")
    i += 1

    while i < 25:
        try:
            timeline_response = api.user_timeline(screen_name = user, count = 25, max_id = last_id)
            n = len(timeline_response)
            last_id = timeline_response[n-1]._json["id"]

            batch_put_response(timeline_response, table)
            i += 1

            if (i > 0 and (i % 5) == 0):
                print("inserted batch {} to dynamodb successfully.".format(i+1))
    
        except Exception as e:
            print(e)
            if "429" in str(e):
                print("sleeping for 5 minutes")
                time.sleep(60*5)

            else:
                print("sleeping for a minute")
                time.sleep(60)

In [28]:
# timeline_response = api.user_timeline(screen_name = my_following[0], count = 100)
# timeline_response[0]._json

## get all tweets

In [29]:
tweets = dynamodb.Table("tweets")
print(tweets)

dynamodb.Table(name='tweets')


In [33]:
try:
    response = tweets.scan()
    response

except Exception as e:
    print(e)

Could not connect to the endpoint URL: "http://xanaxprincess.asuscomm.com:8001/"


In [31]:
my_following

['kittyxcate',
 'coy_valerie23',
 'colborkmundus',
 'poole5nikki',
 'clinton38estell',
 'grace3070stone',
 'una_caraballo40',
 'Jennife45385981',
 'alison6hardin',
 'chastity7godfre',
 'Ashey190',
 'temmyzeus100',
 'packorrodev',
 'iblainee',
 'lykaboss',
 'stephbtechin',
 'annabanana4128',
 '420stoners_Gas',
 'comewhenyoucaII',
 'BloodGoblet',
 'ypurvazha',
 'gigi75119863',
 'HeadlineVC',
 'VivienDrinkwat1',
 'devLabs_',
 'Miranda_Rader97',
 'Marswaggo',
 'DavidMa98889032',
 'Ceresasaurus',
 'boogerscab',
 'setabyelsew',
 'dossermd',
 'cadillac_motel',
 'AbbasyKhan',
 'AdamsSm45393725',
 'very_sad_sab',
 'HunterBernsFndn',
 'SuplexPrincess',
 '_you_like_jazz',
 'porchluck',
 'DonnaHurwitt',
 'fat_clit',
 'gea2006',
 'ChriMoulto',
 '__chelle1',
 'hikikohoney',
 'HawajskaLoreen',
 'SteveDoesnt',
 'betsy_dunn',
 'the1derwood',
 'lizdavas',
 'quinnquan',
 'ghouljrband',
 'rosedustgirl',
 'Kyotolofi',
 'Alfredo15866313',
 'jaqkeystone',
 'poetic_papa',
 'xPOPPYxROSEx',
 'palumbojumbo',
 'm

## get tweets of following

In [15]:
for f in my_following:
    try:
        output = get_tweets(f, tweets)
        time.sleep(5)

    except KeyboardInterrupt as k:
        print(k)

scraping tweets for user: colborkmundus
putting first batch
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfully.
inserted batch to dynamodb successfull

In [11]:
response = tweets.scan()
# response

In [12]:
! aws dynamodb describe-table --table-name "tweets" --endpoint-url http://xanaxprincess.asuscomm.com:8001

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "user",
                "AttributeType": "S"
            },
            {
                "AttributeName": "created_at",
                "AttributeType": "S"
            }
        ],
        "TableName": "tweets",
        "KeySchema": [
            {
                "AttributeName": "user",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "created_at",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1674535588.165,
        "ProvisionedThroughput": {
            "LastIncreaseDateTime": 0.0,
            "LastDecreaseDateTime": 0.0,
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 10
        },
        "TableSizeBytes": 60390,
        "ItemCount": 290,
        "TableArn": "arn:aws:dynamodb:ddblocal:000000000000:t